#### Pair programming Python + SQL


In [1]:
# 1 Importar las librerías necesarias para crear la conexión y trabajar con Pandas.
# Importar librería para la conexión con MySQL
# -----------------------------------------------------------------------
import mysql.connector
from mysql.connector import errorcode


# Importar librerías para manipulación y análisis de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np



In [ ]:
# 2 Crear la conexión a la base de datos Northwind que devuelva un mensaje de error si algo no ocurre cómo debería.


In [ ]:
# 3 Usando "information_schema", extraer los nombres de las columnas de la tabla "customers" de la BBDD "northwind".

In [ ]:
# 4 Seleccionar todo de la tabla "customers" donde "country" sea "USA" o "Germany". Almacenar los resultados en una variable.

In [ ]:
# 5 Convertir los resultados del ejercicio anterior en un Dataframe 
# (no od olvidéis de poner el nombre a las columnas en el orden correcto).

In [ ]:
# 6 Muestrar las 10 primeras filas del Dataframe y las 10 últimas.

In [ ]:
# 7 Mostrar 5 filas aleatorias del Dataframe.

In [ ]:
# 8 ¿Hay nulos en el Dataframe? El resultado debe verse para cada una de las columnas.

In [ ]:
# 9 Mostrar los principales estadísticos.

In [ ]:
# 10 Mostrar el nombre de las columnas del Dataframe.

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
from time import sleep

# Variables with Selector to Use
url_cookies = "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd"
url_search = "#suggestion-search"
url_lookup = "#suggestion-search-button > svg"
url_cast_access = "#fullcredits > span > a"
url_birth_year = "#name-born-info > time"
url_birth_year_exception = "#name-born-info > time"
url_awards = "#awards > span > a"
url_actor_role = "#filmo-head-actor"
url_biography = "#name-bio"

# Definición de funciones
def process_input(file_name):
    df = pd.read_csv(f"api_comedy/{file_name}_API.csv", index_col=0)
    list_of_movies_from_api = list(df.itertuples(index=True, name=None))
    return list_of_movies_from_api

def open_browser(url, driver):
    driver.get(url)
    driver.maximize_window()
    sleep(5)

def accept_cookies(driver, url_cookies):
    try:
        driver.find_element(By.CSS_SELECTOR, url_cookies).click()
    except Exception as e:
        print("Cookies not found or already accepted")
    sleep(3)

def search_movie(driver, url_search, url_lookup, url_cast_access, relation_actor_movie, movie_list):
    full_actor_list = []
    
    for movie_imdb in movie_list:
        print('Película:', movie_imdb)
        
        # Enter search
        search_box = driver.find_element(By.CSS_SELECTOR, url_search)
        search_box.clear()
        search_box.send_keys(movie_imdb[0])
    
        # Click lookup
        driver.find_element(By.CSS_SELECTOR, url_lookup).click()
        
        sleep(5)

        # Access cast
        try: 
            driver.find_element(By.CSS_SELECTOR, url_cast_access).click()
        except Exception as error:
            driver.back()
            continue
            
        id_imdb = movie_imdb[0]
        actor_list = retrieve_actor_data(driver, url_birth_year, url_biography, url_actor_role, url_awards, id_imdb, relation_actor_movie)
            
        full_actor_list.append(actor_list)

    return full_actor_list

def retrieve_actor_data(driver, url_birth_year, url_biography, url_actor_role, url_awards, id_imdb, relation_actor_movie):
    actor_list = []
   
    try: 
        for i in range(2, 7):
            try:
                actor_name = driver.find_element(By.CSS_SELECTOR, f"#fullcredits_content > table.cast_list > tbody > tr:nth-child({i}) > td:nth-child(2) > a").text
                print("Name:", actor_name)
                driver.find_element(By.CSS_SELECTOR, f"#fullcredits_content > table.cast_list > tbody > tr:nth-child({i}) > td:nth-child(2) > a").click()
                sleep(3)
            except Exception as error:
                print("Actor not found") 
                continue

            try:
                actor_birth_year = driver.find_element(By.CSS_SELECTOR, url_birth_year).text
                actor_birth_year = int(actor_birth_year[-4:])
            except Exception as error:
                try:
                    actor_birth_year = driver.find_element(By.CSS_SELECTOR, url_birth_year_exception).text
                    actor_birth_year = int(actor_birth_year[-4:])
                except Exception as error:
                    actor_birth_year = None   
            print("Birth year:", actor_birth_year)

            try:
                driver.find_element(By.CSS_SELECTOR, url_biography).click()
                sleep(2)
            except Exception as error:
                print("Biography not found")

            known_for = []
            try:  
                for j in range(2, 4):
                    known_for.append(driver.find_element(By.CSS_SELECTOR, f"#knownfor > div > ul > li:nth-child({j}) > a").text)
            except Exception as error:
                known_for.append(None)
            print("Know for:", known_for)

            driver.back()
            sleep(2)
            
            actor_role = []
            try:
                actor_role_detail = driver.find_element(By.CSS_SELECTOR, url_actor_role).text
                detail_with_comma = actor_role_detail.replace("\n", ", ")
                actor_role = detail_with_comma.split(", ")
            except Exception as error:
                actor_role.append(None)
            print("Actor role:", actor_role)

            actor_awards = []
            try:
                driver.find_element(By.CSS_SELECTOR, url_awards).click()
                sleep(2)
                for h in range(3, 5):
                    detail = driver.find_element(By.CSS_SELECTOR, f"section.awards > div:nth-child({h}) > h3").text
                    detail_with_comma = detail.replace("\n", ", ")
                    actor_awards.append(detail_with_comma)
                driver.back()
                sleep(2)
            except Exception as error:
                actor_awards.append(None)

            print("Awards:", actor_awards)

            clean_actor = clean_year_know_for(actor_name, actor_birth_year, known_for, actor_role, actor_awards)
            clean_actor_list.append(clean_actor)
            print("Lista actores para dataframe:", clean_actor_list)
            
            try:
                actor_list.append((actor_name, actor_birth_year, known_for, actor_role, actor_awards))
                print("Actor list:", actor_list)
            except Exception as error:
                print("Error append actor list")

            try:
                relation_actor_movie.append([id_imdb, actor_name])
                print("Relation actor movie:", relation_actor_movie)
            except Exception as error:
                print("Error append relation actor movie")

            driver.back()
            sleep(2)
    except Exception as error:
        pass

    return actor_list

def clean_year_know_for(actor_name, actor_birth_year, known_for, actor_role, actor_awards):
    clean_know_for = [titulo.split(' (')[0].replace('"', "").strip() for titulo in known_for if titulo]
    know_for_str = ", ".join(clean_know_for) if clean_know_for else "none"
    role_str = ", ".join(actor_role) if actor_role else "none"
    award_str = ", ".join(actor_awards) if actor_awards else "none"
    actor_birth_year_srt = str(actor_birth_year) if actor_birth_year else "none"
    return (actor_name, actor_birth_year_srt, know_for_str, role_str, award_str)

# Main
file_name = "list_comedy_range_1_2"
list_of_movies_from_api = process_input(file_name)
relation_actor_movie = []
clean_actor_list = []
url = "https://www.imdb.com/"
driver = webdriver.Chrome()
open_browser(url, driver)
accept_cookies(driver, url_cookies)
full_actor_list = search_movie(driver, url_search, url_lookup, url_cast_access, relation_actor_movie, list_of_movies_from_api)
print("Relation actor movie", relation_actor_movie)

# Create DataFrame
headers = ["actor_name", "birth_year", "know_for", "actor_role", "actor_awards"]
df_actor_list = pd.DataFrame(clean_actor_list, columns=headers)
df_actor_list.set_index('actor_name', inplace=True)
print(df_actor_list)

driver.quit()
